# DCC Info Tool

This tool prints the contents of a DCC. By default, it tries to read a QR code from a screenshot. You can modify that behavior if needed. 

It does not verify the signature and is for debugging/analysis only. 

In [4]:
import zlib
import cbor2
import pyautogui
from pprint import pprint
from pyzbar.pyzbar import decode as qr_decode
from base45 import b45decode
from cose.messages import Sign1Message
from datetime import datetime
from dccinfo import get_kid_b64

# Comment the following three lines if you want to enter the QR code data manually 
image = pyautogui.screenshot()
qr_code = qr_decode(image)[0]
qr_code_data =  qr_code.data.decode()

# Uncomment the following line if you want to enter the QR code data manually 
# qr_code_data = 'HC1:...'  
assert qr_code_data.startswith('HC1:')

decompressed = zlib.decompress(b45decode(qr_code_data[4:]))

cose_data = Sign1Message.decode(decompressed)
payload = cbor2.loads(cose_data.payload)                           # Uncomment the corresponding line to...
print('Plain HCERT:\n', qr_code_data, '\n\n')                      # ... print plain HCERT
print('Plain COSE/CBOR data:', cbor2.loads(decompressed), '\n\n')  # ... print plain COSE/CBOR
print(f'Unprotected Header: {cose_data.uhdr}\n')                   # ... print unprotected header
print(f'Protected Header: {cose_data.phdr}\n')                     # ... print protected header
print(f'KID = {get_kid_b64(cose_data)}\n')                         # ... print key ID

try:
    payload[4] = f'Timestamp({datetime.fromtimestamp(payload[4]).isoformat()})'
    payload[6] = f'Timestamp({datetime.fromtimestamp(payload[6]).isoformat()})'
except:
    pass

pprint(payload)


KID = b'p2DmeZjedeA='

{-260: {1: {'dob': '1990-05-06',
            'nam': {'fn': 'JANSEN',
                    'fnt': 'JANSEN',
                    'gn': 'FLOOR',
                    'gnt': 'FLOOR'},
            'r': [],
            't': [],
            'v': [{'ci': 'URN:UVCI:01:GB:4447265FFA5C4B6B88E77CCDA03FD9C0#P',
                   'co': 'GB',
                   'dn': 3,
                   'dt': '2022-01-17',
                   'is': 'NHS Scotland',
                   'ma': 'ORG-100030215',
                   'mp': 'EU/1/20/1528',
                   'sd': 3,
                   'tg': '840539006',
                   'vp': '1119349007'}],
            'ver': '1.3.0'}},
 1: 'GB',
 4: 'Timestamp(2022-01-29T13:36:01)',
 6: 'Timestamp(2022-01-26T13:36:01)'}
